In [166]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

# Non-DataScience/ML packages
import chardet
import re
import string

In [167]:
path="../../../../Datafiles/"
file = 'table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls'
df = pd.read_excel(path + file, header=4)
print("there are {} entries in the data frame".format(len(df)))

there are 351 entries in the data frame


In [168]:
# Let's clean up the column names
colnames = df.columns
newcolnameslist = []

for colname in colnames:
    newcolname=colname.replace('\n','').replace(' ','').capitalize() # strip out newlines, spaces, and captitalize
    newcolname=re.sub(r'\([^)]*\)', '', newcolname)                  # remove parenthesized stuff
    newcolnameslist.append(newcolname)                               # put all the column names into a list 

df.columns = newcolnameslist
print(newcolnameslist)

['City', 'Population', 'Violentcrime', 'Murderandnonnegligentmanslaughter', 'Rape1', 'Rape2', 'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary', 'Larceny-theft', 'Motorvehicletheft', 'Arson3']


In [169]:
df.tail(15)

,City,Population,Violentcrime,Murderandnonnegligentmanslaughter,Rape1,Rape2,Robbery,Aggravatedassault,Propertycrime,Burglary,Larceny-theft,Motorvehicletheft,Arson3
336,Wellsville Village,4574.0,11.0,0.0,NaN,1.0,0.0,10.0,164.0,32.0,129.0,3.0,1.0
337,Westhampton Beach Village,1732.0,2.0,0.0,NaN,0.0,0.0,2.0,44.0,4.0,37.0,3.0,0.0
338,West Seneca Town,44821.0,35.0,1.0,NaN,1.0,9.0,24.0,896.0,166.0,710.0,20.0,0.0
339,Whitehall Village,2604.0,6.0,0.0,NaN,0.0,0.0,6.0,62.0,12.0,49.0,1.0,NaN
340,White Plains,57559.0,78.0,1.0,NaN,3.0,28.0,46.0,1232.0,77.0,1134.0,21.0,NaN
341,Whitesboro Village,3733.0,4.0,0.0,NaN,0.0,0.0,4.0,36.0,14.0,21.0,1.0,0.0
342,Whitestown Town,9141.0,0.0,0.0,NaN,0.0,0.0,0.0,77.0,13.0,61.0,3.0,0.0
343,Woodbury Town,10685.0,3.0,0.0,NaN,0.0,2.0,1.0,541.0,9.0,529.0,3.0,NaN
344,Woodridge Village,829.0,7.0,0.0,NaN,0.0,0.0,7.0,17.0,8.0,9.0,0.0,0.0
345,Woodstock Town,5931.0,2.0,0.0,NaN,0.0,0.0,2.0,58.0,13.0,45.0,0.0,NaN


In [170]:
df[df['City'].str.contains('figures shown')]
df[df['City'].str.contains('publish')]

df = df[(df['City'].str.contains('figures shown') == False)] # We don't want any documentation rows
df = df[(df['City'].str.contains('publish') == False)]       # We don't want any other documentation rows
# All Rape1 values are false
# No Rape2 values are true
# Arson3 NaN's -> 161
# There are 161 NaN entries for Arson3

In [171]:
df['PopulationSquared'] = df.Population.pow(2)
df['Murder'] = np.where(df['Murderandnonnegligentmanslaughter']>=1, 1, 0.0)
df['Murder'] = df['Murder'].astype(int)
df['Robbery'] = np.where(df['Robbery']>=1, 1, 0.0)
df['Robbery'] = df['Robbery'].astype(int)
df.Arson3.fillna(0, inplace=True) # Let's clean up Arson3
df.Arson3 = df.Arson3.astype(int) # Let's make it an int
df = df.rename(columns={'Arson3': 'Arson', 'Rape2': 'Rape', 'Aggravatedassault': 'AggravatedAssault',
                        'Violentcrime':'ViolentCrime','Propertycrime': 'PropertyCrime', 
                        'Larceny-theft':'LaracenyTheft','Motorvehicletheft':'MotorVehicleTheft',
                       'Murderandnonnegligentmanslaughter': 'MurderAndNonNegligentManslaughter'})

In [172]:
# df.head(50)
df.describe()

,Population,ViolentCrime,MurderAndNonNegligentManslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LaracenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
count,3.480000e+02,348.000000,348.000000,0.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,3.480000e+02,348.000000
mean,4.003763e+04,201.594828,1.566092,NaN,5.864943,0.600575,121.261494,792.606322,119.683908,637.017241,35.905172,1.005747,2.035547e+11,0.140805
std,4.500374e+05,2815.268504,18.303673,NaN,60.425452,0.490486,1706.131730,7659.724746,924.948789,6346.054451,403.423826,7.884612,3.778876e+12,0.348321
min,5.260000e+02,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.766760e+05,0.000000
25%,3.003000e+03,2.000000,0.000000,NaN,0.000000,0.000000,1.000000,40.500000,6.000000,31.000000,0.000000,0.000000,9.018117e+06,0.000000
50%,7.233500e+03,6.000000,0.000000,NaN,0.000000,1.000000,4.000000,112.500000,17.500000,94.000000,2.000000,0.000000,5.232568e+07,0.000000
75%,1.842750e+04,22.000000,0.000000,NaN,2.000000,1.000000,14.000000,341.000000,51.250000,287.250000,7.000000,0.000000,3.397536e+08,0.000000
max,8.396126e+06,52384.000000,335.000000,NaN,1112.000000,1.000000,31767.000000,141971.000000,16606.000000,117931.000000,7434.000000,132.000000,7.049493e+13,1.000000


In [173]:
df.describe()

,Population,ViolentCrime,MurderAndNonNegligentManslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LaracenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
count,3.480000e+02,348.000000,348.000000,0.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,3.480000e+02,348.000000
mean,4.003763e+04,201.594828,1.566092,NaN,5.864943,0.600575,121.261494,792.606322,119.683908,637.017241,35.905172,1.005747,2.035547e+11,0.140805
std,4.500374e+05,2815.268504,18.303673,NaN,60.425452,0.490486,1706.131730,7659.724746,924.948789,6346.054451,403.423826,7.884612,3.778876e+12,0.348321
min,5.260000e+02,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.766760e+05,0.000000
25%,3.003000e+03,2.000000,0.000000,NaN,0.000000,0.000000,1.000000,40.500000,6.000000,31.000000,0.000000,0.000000,9.018117e+06,0.000000
50%,7.233500e+03,6.000000,0.000000,NaN,0.000000,1.000000,4.000000,112.500000,17.500000,94.000000,2.000000,0.000000,5.232568e+07,0.000000
75%,1.842750e+04,22.000000,0.000000,NaN,2.000000,1.000000,14.000000,341.000000,51.250000,287.250000,7.000000,0.000000,3.397536e+08,0.000000
max,8.396126e+06,52384.000000,335.000000,NaN,1112.000000,1.000000,31767.000000,141971.000000,16606.000000,117931.000000,7434.000000,132.000000,7.049493e+13,1.000000
